## Setup

### Setup Intel Extensions

In [ ]:
# from sklearnex import patch_sklearn

In [ ]:
# patch_sklearn()

### Predictable randomness

In [ ]:
import numpy as np

seed = 0


def rng():
    return np.random.RandomState(seed)

### Shared parameters

In [ ]:
param_scalers = [None]

### Preprocessing and parameter search

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [ ]:
from molvs import Standardizer
from common.lib.descriptors.cdk import ECFPID, ECFPTransformer

In [ ]:
from common.lib.sklearn import V

In [ ]:
preprocessing_pipeline = Pipeline(
    steps=[
        ("standardizer", FunctionTransformer(V(Standardizer().fragment_parent))),
        ("descriptors", ECFPTransformer(ECFPID.ECFP4)),
    ]
)

In [ ]:
from sklearn.model_selection import KFold, RandomizedSearchCV

In [ ]:
def make_parameter_search(pipeline, cv_params):
    return RandomizedSearchCV(
        # Pipeline(steps=preprocessing_pipeline.steps + pipeline.steps, memory="logs/cache"),
        pipeline,
        cv_params,
        scoring="balanced_accuracy",
        refit=True,
        cv=KFold(n_splits=10, shuffle=True, random_state=seed),
        verbose=3,
        error_score="raise",
        n_jobs=-1,
    )


def parameter_search_name(ps):
    return ps.estimator.steps[-1][0]

### Cross validation

In [ ]:
from sklearn.metrics import get_scorer, make_scorer, precision_score, recall_score
from sklearn.model_selection import cross_validate

In [ ]:
def score(model, X, y, scoring={}):
    return pd.DataFrame({key: [get_scorer(definition)(model, X, y)] for key, definition in scoring.items()})

In [ ]:
def external_validation(model, X, y):
    return score(
        model,
        X,
        y,
        scoring={
            "accuracy": "accuracy",
            "sensitivity": "recall",
            "specificity": make_scorer(recall_score, pos_label=0),
            "balanced_accuracy": "balanced_accuracy",
            "f1": "f1",
            "roc_auc": "roc_auc",
            "precision": make_scorer(precision_score, zero_division=0),
            "matthews_corrcoef": "matthews_corrcoef",
        },
    )

In [ ]:
def cross_validation(model, X, y):
    return cross_validate(
        model,
        X,
        y,
        scoring={
            "accuracy": "accuracy",
            "sensitivity": "recall",
            "specificity": make_scorer(recall_score, pos_label=0),
            "balanced_accuracy": "balanced_accuracy",
            "f1": "f1",
            "roc_auc": "roc_auc",
            "precision": make_scorer(precision_score, zero_division=0),
            "matthews_corrcoef": "matthews_corrcoef",
        },
        cv=KFold(n_splits=10, shuffle=True, random_state=seed),
        n_jobs=-1,
    )

## Define Models

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [ ]:
clf_logr = make_parameter_search(
    Pipeline(
        steps=[
            ("scaler", None),
            ("pca", PCA(n_components=8, random_state=rng())),
            (
                "logr",
                LogisticRegression(solver="saga", max_iter=10000, random_state=rng()),
            ),
        ]
    ),
    {
        "scaler": param_scalers,
        "logr__penalty": ["elasticnet"],
        "logr__C": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
        "logr__l1_ratio": [0, 0.25, 0.5, 0.75, 1],
    },
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_rf = make_parameter_search(
    Pipeline(
        steps=[
            ("scaler", None),
            (
                "rf",
                RandomForestClassifier(max_features=1.0, random_state=rng(), n_jobs=-1),
            ),
        ]
    ),
    {
        "scaler": param_scalers,
        "rf__class_weight": ["balanced"],
        "rf__n_estimators": [
            5,
            10,
            25,
            50,
        ],  # , 100, 250
        "rf__max_depth": [2, 4, 8, 16],  # , 32, 64
    },
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf_knn = make_parameter_search(
    Pipeline(steps=[("scaler", None), ("knn", KNeighborsClassifier())]),
    {
        "scaler": param_scalers,
        "knn__n_neighbors": [3, 5, 9, 11, 13, 17, 19],
        "knn__weights": ["uniform", "distance"],
        "knn__metric": ["euclidean", "manhattan"],
    },
)

In [ ]:
from sklearn.kernel_approximation import Nystroem
from sklearn.svm import SVC

In [ ]:
clf_svc = make_parameter_search(
    Pipeline(
        steps=[
            ("scaler", None),
            ("nystroem", Nystroem()),
            ("svc", SVC(class_weight="balanced", random_state=rng(), probability=True)),
        ]
    ),
    {
        "scaler": param_scalers,
        "nystroem__gamma": [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
        "svc__C": [0.1, 1.0, 10.0, 100.0, 1000.0],
    },
)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
clf_xgb = make_parameter_search(
    Pipeline(
        steps=[
            ("scaler", None),
            ("xgb", XGBClassifier(random_state=rng(), n_jobs=-1)),
        ]
    ),
    {
        "scaler": param_scalers,
        "xgb__scale_pos_weight": [0.1, 0.5, 1, 5, 10],
        "xgb__objective": [None, "binary:logistic"],
        "xgb__n_estimators": [5, 10, 25, 50, 100, 250],
        "xgb__max_depth": [2, 4, 8, 16, 32, 64],
    },
)

In [ ]:
models = [clf_logr, clf_rf, clf_knn, clf_svc, clf_xgb]
targets = [
    "BCRP",
    "BCRP-S",
    "BSEP",
    "MATE1",
    "MDR1",
    "MDR1-S",
    "MRP2-S",
    "MRP3",
    "MRP3-S",
    "OATP1B1",
    "OATP1B3",
    "OCT1",
    "OCT2",
]

## Training

### Training routine

In [ ]:
import pandas as pd
import joblib
from rdkit.Chem.PandasTools import LoadSDF

In [ ]:
def load_data(target):
    data = LoadSDF(
        f"ba_assets/data_for_models/data_threshold_all_filled_0.5_all_masters/training_chembl+manual/{target}.sdf"
    )

    return np.stack(preprocessing_pipeline.transform(data.ROMol)), np.stack(
        data.Classification.astype(int)
    )

In [ ]:
def load_test(target):
    chembl = LoadSDF(
        f"ba_assets/data_for_models/data_threshold_all_filled_0.5_all_masters/testing_chembl/{target}.sdf"
    )
    manual = LoadSDF(
        f"ba_assets/data_for_models/data_threshold_all_filled_0.5_all_masters/testing_manual/{target}.sdf"
    )
    data = pd.concat([chembl, manual])

    return np.stack(preprocessing_pipeline.transform(data.ROMol)), np.stack(
        data.Classification.astype(int)
    )

In [ ]:
def train_model(model, training, test):
    # Destructure data
    X, y = training
    X_test, y_test = test

    # Train
    with joblib.parallel_config("loky", n_jobs=-1):
        model.fit(X, y)

    # Cross validate
    cv = cross_validation(model, X, y)

    # Validate on external data
    external = external_validation(model, X_test, y_test)

    return model, cv, external

### Configure task runner

In [ ]:
import atexit
import shutil

from dask.distributed import Client
from dask_jobqueue import SLURMCluster

In [ ]:
if cluster := globals().get("cluster"):
    cluster.close()
shutil.rmtree("logs", ignore_errors=True)

cluster = SLURMCluster(
    cores=1,
    job_cpu=32,
    memory="8 GB",
    scheduler_options={"interface": "ens9f0", "dashboard_address": ":8787"},
    log_directory="logs",
)

atexit.register(lambda: cluster.close())

In [ ]:
cluster.scale(len(models) * len(targets))

In [ ]:
client = Client(cluster)

### Start jobs

In [ ]:
data_training = {target: client.submit(load_data, target, priority=1) for target in targets}
data_test = {target: client.submit(load_test, target, priority=1) for target in targets}

In [ ]:
jobs = {
    (model_name, target_name): client.submit(
        train_model,
        model,
        data_training[target_name],
        data_test[target_name],
        key=f"train_model_{model_name}-{target_name}",
    )
    for model_name, model in ((parameter_search_name(model), model) for model in models)
    for target_name in targets
}

In [ ]:
# [job.result() for job in jobs.values() if job.done()]

### Write out models

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
model_dir = Path("models")

In [ ]:
def name_to_path(name):
    target, *maybe_substrate, architecture = name.split("-")

    if maybe_substrate == ["S"]:
        return Path(f"Substrate/{target}/{architecture}_clf")
    else:
        return Path(f"Inhibition/{target}/{architecture}_clf")

In [ ]:
results = [(name, job.result()) for name, job in jobs.items() if job.done()]


In [ ]:
results = [(name, job.result()) for name, job in jobs.items() if job.done()]

for name, (model, cv, external) in results:
    path = name_to_path(name)
    model_dir.joinpath(path).parent.mkdir(exist_ok=True, parents=True)

    joblib.dump(model, model_dir.joinpath(f"{path}.pkl"))
    pd.DataFrame(cv).to_csv(model_dir.joinpath(f"{path}.csv"), index=False)
    pd.DataFrame(external).to_csv(model_dir.joinpath(f"{path}.external.csv"), index=False)

jobs_count, done_count = len(jobs), len([job for job in jobs.values() if job.done()])

print(f"Saved {len(results)}/{len(jobs)} models")

In [ ]:
[job.result() for name, job in jobs.items() if job.done() and name.endswith("float64")]